# Introduction

Since Jan. 1, 2015, [The Washington Post](https://www.washingtonpost.com/) has been compiling a database of every fatal shooting in the US by a police officer in the line of duty. 

<center><img src=https://i.imgur.com/sX3K62b.png></center>

While there are many challenges regarding data collection and reporting, The Washington Post has been tracking more than a dozen details about each killing. This includes the race, age and gender of the deceased, whether the person was armed, and whether the victim was experiencing a mental-health crisis. The Washington Post has gathered this supplemental information from law enforcement websites, local new reports, social media, and by monitoring independent databases such as "Killed by police" and "Fatal Encounters". The Post has also conducted additional reporting in many cases.

There are 4 additional datasets: US census data on poverty rate, high school graduation rate, median household income, and racial demographics. [Source of census data](https://factfinder.census.gov/faces/nav/jsf/pages/community_facts.xhtml).

## Import Statements

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

## Notebook Presentation

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

## Load the Data

In [3]:
df_hh_income = pd.read_csv('Median_Household_Income_2015.csv', encoding="windows-1252")
df_pct_poverty = pd.read_csv('Pct_People_Below_Poverty_Level.csv', encoding="windows-1252")
df_pct_completed_hs = pd.read_csv('Pct_Over_25_Completed_High_School.csv', encoding="windows-1252")
df_share_race_city = pd.read_csv('Share_of_Race_By_City.csv', encoding="windows-1252")
df_fatalities = pd.read_csv('Deaths_by_Police_US.csv', encoding="windows-1252")

# Preliminary Data Exploration

* What is the shape of the DataFrames? 
* How many rows and columns do they have?
* What are the column names?
* Are there any NaN values or duplicates?

In [4]:
print(f"The shape of df_hh_income is: {df_hh_income.shape}")
print(f"The shape of df_pct_poverty is: {df_pct_poverty.shape}")
print(f"The shape of df_pct_completed_hs is: {df_pct_completed_hs.shape}")
print(f"The shape of df_share_race_city is: {df_share_race_city.shape}")
print(f"The shape of df_fatalities is: {df_fatalities.shape}")

The shape of df_hh_income is: (29322, 3)
The shape of df_pct_poverty is: (29329, 3)
The shape of df_pct_completed_hs is: (29329, 3)
The shape of df_share_race_city is: (29268, 7)
The shape of df_fatalities is: (2535, 14)


In [5]:
print(f"The columns' name of df_hh_income are: {df_hh_income.columns}")
print(f"The columns' name of df_pct_poverty are: {df_pct_poverty.columns}")
print(f"The columns' name of df_pct_completed_hs are: {df_pct_completed_hs.columns}")
print(f"The columns' name of df_share_race_city are: {df_share_race_city.columns}")
print(f"The columns' name of  are: {df_fatalities.columns}")

The columns' name of df_hh_income are: Index(['Geographic Area', 'City', 'Median Income'], dtype='object')
The columns' name of df_pct_poverty are: Index(['Geographic Area', 'City', 'poverty_rate'], dtype='object')
The columns' name of df_pct_completed_hs are: Index(['Geographic Area', 'City', 'percent_completed_hs'], dtype='object')
The columns' name of df_share_race_city are: Index(['Geographic area', 'City', 'share_white', 'share_black',
       'share_native_american', 'share_asian', 'share_hispanic'],
      dtype='object')
The columns' name of  are: Index(['id', 'name', 'date', 'manner_of_death', 'armed', 'age', 'gender',
       'race', 'city', 'state', 'signs_of_mental_illness', 'threat_level',
       'flee', 'body_camera'],
      dtype='object')


In [6]:
print(f"Is there any NaN or Duplicated values in df_hh_income: NaN:{df_hh_income.isna().values.any()}, Duplicated:{df_hh_income.duplicated().values.any()}")
print(f"Is there any NaN or Duplicated values in df_pct_poverty: NaN:{df_pct_poverty.isna().values.any()}, Duplicated:{df_pct_poverty.duplicated().values.any()}")
print(f"Is there any NaN or Duplicated values in df_pct_completed_hs: NaN:{df_pct_completed_hs.isna().values.any()}, Duplicated:{df_pct_completed_hs.duplicated().values.any()}")
print(f"Is there any NaN or Duplicated values in df_share_race_city: NaN:{df_share_race_city.isna().values.any()}, Duplicated:{df_share_race_city.duplicated().values.any()}")
print(f"Is there any NaN or Duplicated values in df_fatalities: NaN:{df_fatalities.isna().values.any()}, Duplicated:{df_fatalities.duplicated().values.any()}")

Is there any NaN or Duplicated values in df_hh_income: NaN:True, Duplicated:False
Is there any NaN or Duplicated values in df_pct_poverty: NaN:False, Duplicated:False
Is there any NaN or Duplicated values in df_pct_completed_hs: NaN:False, Duplicated:False
Is there any NaN or Duplicated values in df_share_race_city: NaN:False, Duplicated:False
Is there any NaN or Duplicated values in df_fatalities: NaN:True, Duplicated:False


## Data Cleaning - Check for Missing Values and Duplicates

Consider how to deal with the NaN values. Perhaps substituting 0 is appropriate. 

In [7]:
df_hh_income["Median Income"] = df_hh_income["Median Income"].replace(["(X)", "-", "N/A", ""], np.nan) # change the illegal sign to np.nan
df_hh_income = df_hh_income.fillna(0)
df_hh_income.isna().values.any()

np.False_

In [8]:
df_fatalities["race"] = df_fatalities["race"].fillna("Missing")
df_fatalities["age"] = df_fatalities["age"].fillna(0)
df_fatalities["flee"] = df_fatalities["flee"].fillna("Unknown")
df_fatalities["armed"] = df_fatalities["armed"].fillna("UNknown")
df_fatalities.isna().values.any()

np.False_

# Chart the Poverty Rate in each US State

Create a bar chart that ranks the poverty rate from highest to lowest by US state. Which state has the highest poverty rate? Which state has the lowest poverty rate?  Bar Plot

In [25]:
df_pct_poverty["poverty_rate"] = df_pct_poverty.poverty_rate.replace("-", 0) # replace the "-" data to 0
df_pct_poverty.poverty_rate = pd.to_numeric(df_pct_poverty.poverty_rate) # Turn the poverty_rate to numeric type
df_poverty_rate = df_pct_poverty.groupby("Geographic Area").agg({"poverty_rate": pd.Series.mean}).reset_index()
df_poverty_rate = df_poverty_rate.sort_values(by="poverty_rate", ascending=False)
df_poverty_rate

,Geographic Area,poverty_rate
25,MS,26.88
3,AZ,25.27
10,GA,23.66
2,AR,22.96
32,NM,22.51
18,LA,22.29
40,SC,22.11
49,WV,21.08
1,AL,20.61
36,OK,20.55


In [ ]:
# Matplotlib and Plotly can not be used at the same time.

fig = px.bar(
    data_frame=df_poverty_rate,
    x="Geographic Area",
    y="poverty_rate",
    color="poverty_rate",
    title="The Poverty Rate in Each US State"
)

fig.update_layout(
    xaxis_title="States",
    yaxis_title="Poverty Rate"
)

fig.show()

# Chart the High School Graduation Rate by US State

Show the High School Graduation Rate in ascending order of US States. Which state has the lowest high school graduation rate? Which state has the highest?

In [35]:
df_pct_completed_hs.percent_completed_hs = df_pct_completed_hs.percent_completed_hs.replace("-", 0)
df_pct_completed_hs.percent_completed_hs = pd.to_numeric(df_pct_completed_hs.percent_completed_hs)
df_pct_graduation_by_states = df_pct_completed_hs.groupby("Geographic Area").agg({"percent_completed_hs": pd.Series.mean}).reset_index()
df_pct_graduation_by_states = df_pct_graduation_by_states.sort_values("percent_completed_hs", ascending=False)
df_pct_graduation_by_states

,Geographic Area,percent_completed_hs
19,MA,92.03
11,HI,91.67
6,CT,91.59
21,ME,91.43
30,NH,90.71
34,NY,90.54
31,NJ,90.52
48,WI,90.26
26,MT,89.99
46,VT,89.98


In [37]:
fig = px.bar(
    data_frame=df_pct_graduation_by_states,
    x="Geographic Area",
    y="percent_completed_hs",
    color="percent_completed_hs",
    title="High School Graduation Rate by US State"
)

fig.update_layout(
    xaxis_title="States",
    yaxis_title="Graduation Rate"
)

fig.show()

# Visualise the Relationship between Poverty Rates and High School Graduation Rates

#### Create a line chart with two y-axes to show if the rations of poverty and high school graduation move together.  

#### Now use a Seaborn .jointplot() with a Kernel Density Estimate (KDE) and/or scatter plot to visualise the same relationship

#### Seaborn's `.lmplot()` or `.regplot()` to show a linear regression between the poverty ratio and the high school graduation ratio. 

# Create a Bar Chart with Subsections Showing the Racial Makeup of Each US State

Visualise the share of the white, black, hispanic, asian and native american population in each US State using a bar chart with sub sections. 

# Create Donut Chart by of People Killed by Race

Hint: Use `.value_counts()`

# Create a Chart Comparing the Total Number of Deaths of Men and Women

Use `df_fatalities` to illustrate how many more men are killed compared to women. 

# Create a Box Plot Showing the Age and Manner of Death

Break out the data by gender using `df_fatalities`. Is there a difference between men and women in the manner of death? 

# Were People Armed? 

In what percentage of police killings were people armed? Create chart that show what kind of weapon (if any) the deceased was carrying. How many of the people killed by police were armed with guns versus unarmed? 

# How Old Were the People Killed?

Work out what percentage of people killed were under 25 years old.  

Create a histogram and KDE plot that shows the distribution of ages of the people killed by police. 

Create a seperate KDE plot for each race. Is there a difference between the distributions? 

# Race of People Killed

Create a chart that shows the total number of people killed by race. 

# Mental Illness and Police Killings

What percentage of people killed by police have been diagnosed with a mental illness?

# In Which Cities Do the Most Police Killings Take Place?

Create a chart ranking the top 10 cities with the most police killings. Which cities are the most dangerous?  

# Rate of Death by Race

Find the share of each race in the top 10 cities. Contrast this with the top 10 cities of police killings to work out the rate at which people are killed by race for each city. 

# Create a Choropleth Map of Police Killings by US State

Which states are the most dangerous? Compare your map with your previous chart. Are these the same states with high degrees of poverty? 

# Number of Police Killings Over Time

Analyse the Number of Police Killings over Time. Is there a trend in the data? 

# Epilogue

Now that you have analysed the data yourself, read [The Washington Post's analysis here](https://www.washingtonpost.com/graphics/investigations/police-shootings-database/).